In [24]:
def get_GPUInfo(i=None):
    from crimm.Utils import cuda_info
    from crimm.Utils.cuda_info import CUDAInfo
    device_info = CUDAInfo()
    if i == None:
        for i in range(device_info.n_gpus):
            print(device_info.devices[i].name)
    else:
        return device_info.devices[i].name
    return

In [1]:
import os
import time
import numpy as np
import yaml
#print(os.environ['CHARMM_LIB_DIR'])
#benchdir = '2_T4L/charmm/template'
#benchdir = '2_T4L/bladelib/template'
#benchdir = '4_HSP90/charmm/template'
#benchdir = '4_HSP90/bladelib/template'
#benchdir = '5_newDHFR/openmm'
#benchdir = '5_newDHFR/bladelib'
#benchdir = '6_dmpg/bladelib'
#benchdir = '6_dmpg/openmm'
#benchdir = 'apoa1_bench/openmm'
#benchdir = 'apoa1_bench/bladelib'
benchdir = 'stmv_bench/openmm'
#benchdir = 'stmv_bench/bladelib'
home = os.getcwd()
# Read the benchmark data from yaml file
yi = open(f'{benchdir}/bench.yml','r')
input_ = yaml.safe_load(yi)
print(input_)
yi.close()

{'system': 'stmv_system', 'platform': 'openmm', 'ngpus': 1, 'benchmark': {'niter': 2}, 'dyn': {'nprint': 1000, 'nsteps': 5000, 'time_step': 0.004, 'temp': 298.15, 'leap': True, 'lang': True, 'cpt': False, 'pconstant': False, 'pmass': False, 'pref': False, 'pgamma': False, 'hoover': False, 'tmass': False, 'nsavl': 0, 'ntrfrq': 0, 'useblade': False, 'useomm': 'gamma 2 prmc pref 1 iprsfrq 100', 'usedomdec': False}, 'files': {'crd': 'stmv_system.crd', 'rtf': ['toppar/top_all36_prot.rtf', 'toppar/top_all36_na.rtf'], 'pdb': '', 'psf': 'stmv_system_hmr.psf', 'param': [['toppar/par_all36m_prot.prm', 'toppar/par_all36_na.prm'], True], 'stream': ['toppar/toppar_water_ions.str']}, 'nnb': {'ctofnb': 9.0, 'ctonnb': 9.0, 'cutim': 9.0, 'cutnb': 9.0, 'fftx': 216, 'ffty': 216, 'fftz': 216, 'kappa': 0.34, 'vfswit': False, 'vswit': True}, 'pbc': {'xtltyp': 'cubic', 'boxsize': [216.832, 216.832, 216.832], 'segments': []}}


In [3]:
import pycharmm_init
pycharmm_init.chsize = 1500000
from pycharmm import *
import pycharmm

In [4]:
def setup_nonbond(cutnb=9.0, ctofnb=9.0, ctonnb=9.0,
                  vswit=True, vfswit=False, kappa=0.32,
                  fftx=64,ffty=64,fftz=64):
    script.CommandScript('faster', on=True).run()
    nbond = pycharmm.NonBondedScript(cutnb=cutnb, cutim=cutnb,
                                       ctonnb=ctonnb, ctofnb=ctofnb,
                                       bycb=True,cdie=True, eps=1,
                                       atom=True, vatom=True,
                                       switch=True, vfswit=vfswit, vswit=vswit,
                                       inbfrq=-1, imgfrq=-1,
                                       ewald=True, pmewald=True, kappa=kappa,
                                       fftx=fftx,ffty=ffty,fftz=fftz,order=4)
    
    return nbond

def setup_PBC(boxsize=[], cutoff=9.0, segments=[],
              xtltyp='cubic', blade=False):
    from pycharmm import crystal, image, coor
    import numpy as np
    """input: boxhalf [0.0]
              cutoff [9.0]
              segments  []
              blade [False]
    defines the periodic boundary conditions for a cubic volume of boxsize. 
    Uses: crystal_define_cubic(), crystal.build(), image.setup_residue,
    image.setup_segment to construct symmetry operations. 

    If global variable openmm is true
    the image centering is at [boxhalf,boxhalf,boxhalf] otherwise at [0,0,0].
    """
    if xtltyp == 'ortho': crystal.define_ortho(boxsize[0],boxsize[1],boxsize[2])
    elif xtltyp == 'cubic': crystal.define_cubic(boxsize[0])
    crystal.build(cutoff)

    if len(segments)>0:
        # This is unnessary for GPU benchmarks
        res_sel = pycharmm.SelectAtoms(select_all=True)
        for segid in segments:
            res_sel = res_sel & ~pycharmm.SelectAtoms(seg_id=segid)
        resnames = list(set(pycharmm.SelectAtoms(res_sel).get_res_names()))

        for segment in segments:
            image.setup_segment(0.0, 0.0, 0.0, segment)
        for residue in resnames:
            image.setup_residue(0.0, 0.0, 0.0, residue)
        if not blade:
            pos = coor.get_positions()
            pos.x += boxsize[0]/2
            pos.y += boxsize[1]/2
            pos.z += boxsize[2]/2
            coor.set_positions(pos)

    return

def run_md(useomm=False,useblade=False,nsteps=50000,nprint=10000,time_step=0.002,
           leap=True,lang=True, cpt=False, temp=298.15, pconstant=False, pmass=False,
           pref=False, pgamma=False, hoover=False, tmass=False, nsavl=0, ntrfrq=0):
    
    dynamics.set_fbetas(np.full(psf.get_natom(),1.0,dtype=float))
    # open unit 3 write form name lambda.lam
    lam_file = pycharmm.CharmmFile(file_name='lambda.lmd', 
                                   file_unit=3,
                                   formatted=False,read_only=False)
   
    my_dyn = pycharmm.DynamicsScript(leap=leap, cpt=cpt, lang=lang, start=True,
                                     nstep=nsteps, timest=time_step,
                                     firstt=temp, finalt=temp, tbath=temp,
                                     tstruc=temp, reft=temp,
                                     teminc=0.0, twindh=0.0, twindl=0.0,
                                     pconstant=pconstant, pmass=pmass, pref=pref,
                                     pgamma=pgamma, hoover=hoover, tmass=tmass,
                                     inbfrq=0, imgfrq=0,  # on GPUs these can be zero
                                     iasors=0, iasvel=1, ichecw=0, iscale=0,
                                     iscvel=0, echeck=-1, nsavc=0, nsavv=0, nsavl=nsavl, ntrfrq=ntrfrq,
                                     iprfrq=2*nprint, nprint=nprint, ihtfrq=0, ieqfrq=0,
                                     ilbfrq=0, ihbfrq=0,iunlam=lam_file.file_unit,
                                     omm=useomm, blade=useblade )
    my_dyn.run()
    lam_file.close()

    return

def setup_system(rtf=[],param=[],pflex=False,msld=False,msld_variables='',stream=[],psf='',crd='',pdb=''):
    # set bomlev to -1 incase error in NBFIX stuff
    obl = settings.set_bomb_level(-1)
    if len(rtf)>0:
        append=False
        for i, top in enumerate(rtf):
            if i > 0: append=True
            read.rtf(top,append=append)
    
    if len(param)>0:
        append=False
        for i, par in enumerate(param):
            if i > 0: append=True
            read.prm(par,append=append,flex=pflex)
    if msld: read.stream(msld_variables) 
    if len(stream)>0:
        for f in stream:
            read.stream(f)
    settings.set_bomb_level(obl)
    if len(psf)>0: read.psf_card(psf)
    if len(crd)>0: read.coor_card(crd)
    if len(pdb)>0: read.pdb(pdb,resid=True)
    return

In [5]:
def hmr(newpsf=''):
    # HMR revisited
    # Get all the masses from the current atoms
    masses = np.array(psf.get_amass())
    resnames = np.array(atom_info.get_res_names(np.arange(0,psf.get_natom(),1)))
    # Build a logical array of all the atoms which are not 'TIP3' waters
    not_waters = resnames != 'TIP3'
    # Build a logical array of all hydrogen atoms based on criterion m_H <= 2
    hydrogens = masses <= 2
    # Build a logical array of all hydrogen atoms not belonging to 'TIP3' waters
    not_water_hydrogen = hydrogens*not_waters
    # Augment each non-water hydrogen mass by 2x original hydrogen mass
    masses += 2*masses*not_water_hydrogen
    # Process the bond array to find heavy atom - hydrogen bonds, 
    # reduce mass of heavy atom by 2*m_H
    bonds = np.array(psf.get_ib_jb())
    for ibnd in range(bonds.shape[1]):
        ib = bonds[0,ibnd]-1
        jb = bonds[1,ibnd]-1
        if not_water_hydrogen[ib] or not_water_hydrogen[jb]:
            if not_water_hydrogen[ib]: masses[jb] -= 2.016
            else: masses[ib] -= 2.016
    # Reset masses to new values
    scalar.set_masses(masses)
    # Write the new psf if requested
    if newpsf != '': write.psf_card(newpsf)
    return

In [6]:
os.chdir(benchdir)
psf_file = input_['files']['psf']

msld = 'msld' in input_.keys()
if (input_['dyn']['time_step'] > 0.002 and\
   os.path.isfile(f'{input_["system"]}_hmr.psf')) and\
   not msld:
    psf_file = f'{input_["system"]}_hmr.psf'

if msld:
    msld_variables = input_['msld']['variables_file']
    lingo.set_charmm_variable('FNEX',5.5)
    lingo.set_charmm_variable('TEMP',298.15)
else: msld_variables = ''
param1 = ''
if len(input_['files']['param']) > 1:
    param0 = input_['files']['param'][0]
    param1 = input_['files']['param'][1]
elif len(input_['files']['param']) == 1:
    param0 = input_['files']['param'][0]
else:
    param0 = ''
    param1 = ''
setup_system(rtf=input_['files']['rtf'],
             param=param0,
             pflex=param1,
             msld=msld,
             msld_variables=msld_variables,
             stream=input_['files']['stream'],
             psf=psf_file,
             crd=input_['files']['crd'],
             pdb=input_['files']['pdb'])

if input_['dyn']['time_step'] > 0.002 and\
  not os.path.isfile(f'{input_["system"]}_hmr.psf'):
    hmr(newpsf=f'{input_["system"]}_hmr.psf')    

  
 CHARMM>     read rtf card -
 CHARMM>     name toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name toppar/top_all36_na.rtf -
 CHARMM>     append
 VOPEN> Attempting to open::toppar/top_all36_na.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  \\\\ CHARMM36 ALL-HYDROGEN NUCLEIC ACID TOPOLOGY FILE ////
 TITLE> *   ALEXANDER D. MACKERELL JR. AND COWORKERS
 TITLE> *                      APRIL 2011
 TITLE> * ALL COMMENTS TO T

 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


In [7]:
locals().update(input_['nnb'])
nnb = setup_nonbond(cutnb=cutnb, ctofnb=ctofnb, ctonnb=ctonnb,
                    vswit=vswit, vfswit=vfswit,
                    kappa=kappa, fftx=fftx, ffty=ffty, fftz=fftz)
nnb.run()

# for benchmarks on GPUs using BLaDE and OpenMM APIs image centering not required
# thus setting empty segments and resnames list acheive this
# This is done in stream file for msld benchmarks
if not 'msld' in input_.keys(): 
    setup_PBC(boxsize=input_['pbc']['boxsize'],xtltyp=input_['pbc']['xtltyp'],
          segments=[], blade=(input_['platform'] == 'blade'))

shake.on(param=True,fast=True,tol=1e-7,bonh=True)
if input_['platform'] == 'openmm' and input_['ngpus'] == 2:
    script.CommandScript('omm',deviceid='0,1').run()
elif input_['platform'] == 'domdec':
    lingo.set_charmm_variable('NNODES',input_['ngpus'])    
    script.CommandScript('domdec', gpu='only', dlb='off', ndir='1 @nnodes 1').run()
timing = []
locals().update(input_['dyn'])
if pmass:
    pmass = psf.get_natom() * 0.12
for run in range(input_['benchmark']['niter']):
    timing.append(time.time())
    run_md(useomm=useomm,useblade=useblade,
           nsteps=nsteps,nprint=nprint,
           time_step=time_step,
           leap=leap,lang=lang,temp=temp, pconstant=pconstant, pmass=pmass,
           pref=pref, pgamma=pgamma, hoover=hoover, tmass=tmass, nsavl=nsavl,
           ntrfrq=ntrfrq)
    timing[-1] = time.time() - timing[-1]

timing = 1.0/np.asarray(timing) # nsteps/sec
timing *= (nsteps*time_step)*(60*60*24)/(1000000.0*0.001)
print(f'Timing {np.mean(timing):.2f} +/- {np.std(timing):.2f} ns/day')

  
 CHARMM>     faster on
 MISCOM> FAST option: EXPANDED (limited fast routines)
  
 CHARMM>     
  
  
 CHARMM>     nbonds cutnb 9.0 -
 CHARMM>     cutim 9.0 -
 CHARMM>     ctonnb 9.0 -
 CHARMM>     ctofnb 9.0 -
 CHARMM>     bycb -
 CHARMM>     cdie -
 CHARMM>     eps 1 -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     switch -
 CHARMM>     vswit -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1 -
 CHARMM>     ewald -
 CHARMM>     pmewald -
 CHARMM>     kappa 0.34 -
 CHARMM>     fftx 216 -
 CHARMM>     ffty 216 -
 CHARMM>     fftz 216 -
 CHARMM>     order 4
 <PME> Total heap storage needed =    23115095
 Fill Ewald table: Number of points=     10000 EWXmax=    3.230000
 fill erfc table: linear inter has rms error = 0.648784D-08 maximum error = 0.126344D-07
 fill erfc table: cubic spline has rms error = 0.181286D-11 maximum error = 0.476019D-11

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  SWITch   VATOm    VSWItch 
     BYCB     NOEXtnd  EWALd   
 CUTNB  =  9

get_PlatformDefaults> Default values found: platform=CUDA precision=single deviceid=0
Setup_OpenMM: Initializing OpenMM context
CHARMM> Using OpenMM functionality for electrostatic and vdW interactions exclusively
CHARMM> Using OpenMM nb routines.
CHARMM> configuring OpenMM PME with
        alpha     0.3400
        fft  216  216  216

NBFIxes in effect, using CustomNonbondedForce
CHARMM> OpenMM switching function selection.
CHARMM> Electrostatic options: switch=F fswitch=F fshift=F PME/Ewald=T RxnFld=F
CHARMM> van der Waals options: vswitch=F vfswitch=F OpenMM vdW switch=F BLOCK vdW softcore:F
Init_context: Using OpenMM platform CUDA      
       CudaDeviceIndex = 0
       CudaCompiler = /usr/local/cuda/bin/nvcc
       CudaPrecision = single
OpenMM: Velocities scaled or randomized
DYNA DYN: Step         Time      TOTEner        TOTKe       ENERgy  TEMPerature
DYNA PROP:             GRMS      HFCTote        HFCKe       EHFCor        VIRKe
DYNA INTERN:          BONDs       ANGLes       U

 VOPEN> Attempting to open::lambda.lmd::
  
 CHARMM>     dynamics leap -
 CHARMM>     lang -
 CHARMM>     start -
 CHARMM>     nstep 5000 -
 CHARMM>     timest 0.004 -
 CHARMM>     firstt 298.15 -
 CHARMM>     finalt 298.15 -
 CHARMM>     tbath 298.15 -
 CHARMM>     tstruc 298.15 -
 CHARMM>     reft 298.15 -
 CHARMM>     teminc 0.0 -
 CHARMM>     twindh 0.0 -
 CHARMM>     twindl 0.0 -
 CHARMM>     inbfrq 0 -
 CHARMM>     imgfrq 0 -
 CHARMM>     iasors 0 -
 CHARMM>     iasvel 1 -
 CHARMM>     ichecw 0 -
 CHARMM>     iscale 0 -
 CHARMM>     iscvel 0 -
 CHARMM>     echeck -1 -
 CHARMM>     nsavc 0 -
 CHARMM>     nsavv 0 -
 CHARMM>     nsavl 0 -
 CHARMM>     ntrfrq 0 -
 CHARMM>     iprfrq 2000 -
 CHARMM>     nprint 1000 -
 CHARMM>     ihtfrq 0 -
 CHARMM>     ieqfrq 0 -
 CHARMM>     ilbfrq 0 -
 CHARMM>     ihbfrq 0 -
 CHARMM>     iunlam 3 -
 CHARMM>     omm gamma 2 prmc pref 1 iprsfrq 100
  IUNREA = -1         IUNWRI = -1          IUNOS = -1
  IUNCRD = -1         IUNVEL = -1          KUNIT 